In [1]:
!pip install transformers peft accelerate streamlit pyngrok sentencepiece bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 69.0 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%%writefile model_loader.py
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
import torch

BASE_MODEL = "mistralai/Mistral-7B-v0.1"
LORA_PATH = "/content/drive/MyDrive/mistral-finetuned"

def load_model():
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=False,
    )

    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
    tokenizer.pad_token = tokenizer.eos_token

    base = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL,
        quantization_config=bnb_config,
        device_map="auto",
    )

    model = PeftModel.from_pretrained(
        base,
        LORA_PATH,
    )

    return model, tokenizer


Writing model_loader.py


In [4]:
%%writefile app.py
import streamlit as st
import torch
from model_loader import load_model

@st.cache_resource
def get_model_and_tokenizer():
    return load_model()

st.title("Mistral Code Optimizer – Fine-tuned Model")

st.write("Enter your code to optimize:")

model, tokenizer = get_model_and_tokenizer()

user_input = st.text_area("Your code / instruction", height=200)

if st.button("Generate"):
    with st.spinner("Optimizing..."):

        prompt = f"<s>[INST] {user_input} [/INST]"

        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

        outputs = model.generate(
            **inputs,
            max_new_tokens=512,
            temperature=0.3,
        )

        result = tokenizer.decode(outputs[0], skip_special_tokens=True)
        result = result.replace(prompt, "").strip()

        st.write("### Optimized Code:")
        st.code(result)


Writing app.py


In [35]:
# FIX: Manual Download and Installation of ngrok v3
import os
import shutil
from pyngrok import conf, ngrok

# 1. Download the ngrok v3 Linux AMD64 binary (.tgz file)
# This is a stable link provided in the official ngrok documentation.
print("Downloading ngrok binary...")
!wget -q https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz

# 2. Extract the binary and move it to a system path
if os.path.exists("ngrok-v3-stable-linux-amd64.tgz"):
    print("Extracting ngrok binary...")
    # Use 'tar -xzf' to correctly extract the .tgz file
    !tar -xzf ngrok-v3-stable-linux-amd64.tgz

    # Check if extraction was successful and move the binary
    if os.path.exists("ngrok"):
        !mv ngrok /usr/local/bin/ngrok
        !chmod +x /usr/local/bin/ngrok
        print("✅ Success: ngrok v3 is now installed and executable.")

        # 3. Configure pyngrok to use this manual installation
        # This is the critical step to prevent pyngrok from trying (and failing) to download it again.
        conf.get_default().ngrok_path = "/usr/local/bin/ngrok"

        print("\nPyngrok is configured to use the manual installation.")
        !ngrok --version
    else:
        print("❌ Error: The file was downloaded, but the 'ngrok' binary was not found after extraction.")
else:
    print("❌ Error: Download failed. The file was not found at the source URL.")

Extracting ngrok binary...
✅ Success: ngrok v3 is now installed and executable.

Pyngrok is configured to use the manual installation.
ngrok version 3.33.1


In [ ]:
import os
from pyngrok import ngrok
import subprocess
import time
import sys

NGROK_AUTH_TOKEN = "36117wvAoozGww5SI1qjgJDptFx_2uRizsKgKFFMaxkq6wEQT"
STREAMLIT_PORT = 8501
STREAMLIT_APP = "app.py"


print("Killing all existing ngrok tunnels...")
ngrok.kill()

print(f"Checking for and killing any process on port {STREAMLIT_PORT}...")
try:
    subprocess.run(
        ["fuser", "-k", f"{STREAMLIT_PORT}/tcp"],
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL,
        check=False
    )
    time.sleep(1)
    print("Port cleared.")
except Exception as e:
    print(f"Could not clear port: {e}")

try:
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
except Exception as e:
    print(f"Error setting ngrok authtoken: {e}")
    sys.exit(1)

print(f"Starting Streamlit app '{STREAMLIT_APP}' on port {STREAMLIT_PORT}...")
streamlit_command = [
    "streamlit", "run", STREAMLIT_APP,
    "--server.port", str(STREAMLIT_PORT),
    "--server.headless", "true"
]

streamlit_process = subprocess.Popen(
    streamlit_command,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True
)
print(f"Streamlit process started with PID: {streamlit_process.pid}")

time.sleep(10)

if streamlit_process.poll() is not None:
    print("\n--- Streamlit App Failed to Start ---")
    print(f"Process exited with return code: {streamlit_process.returncode}")
    stdout, stderr = streamlit_process.communicate()
    if stdout:
        print("\n[STDOUT]")
        print(stdout)
    if stderr:
        print("\n[STDERR]")
        print(stderr)
    print("--------------------------------------\n")
    print("The Streamlit app crashed. Please check the [STDERR] output above for the cause.")
    streamlit_process.terminate()
else:
    try:
        print("Streamlit process is running. Creating ngrok tunnel...")
        public_url = ngrok.connect(STREAMLIT_PORT).public_url
        print(f"\nYour Streamlit app is publicly available at: {public_url}")
        print("The tunnel will remain open until you stop this cell execution.")

        while True:
            if streamlit_process.poll() is not None:
                print("\n--- Streamlit App CRASHED AFTER STARTUP ---")
                print("The Streamlit process terminated unexpectedly. Check the logs for details.")
                break
            time.sleep(1)

    except Exception as e:
        print(f"\nAn error occurred while setting up ngrok: {e}")
        streamlit_process.terminate()

ngrok.kill()

Killing all existing ngrok tunnels...
Checking for and killing any process on port 8501...
Port cleared.
Starting Streamlit app 'app.py' on port 8501...
Streamlit process started with PID: 6752
Streamlit process is running. Creating ngrok tunnel...

Your Streamlit app is publicly available at: https://opinionatedly-claviform-renata.ngrok-free.dev
The tunnel will remain open until you stop this cell execution.
